From Wageningen B-series open water propeller performance charts (https://doi.org/10.5281/zenodo.8352831):

As early as 1883, R.E. Froude, after extensive experiments with model propellers behind ship models, and model propellers separate from the ship, found that a propeller behind a ship, rotating at a rotational velocityn , behaves like a propeller separate from the ship, operating at the same rotational velocityn , at a reduced advance velocity $V_0 < V$ , called ‘effective advance velocity’. He then introduced the ‘effective wake fraction’w and the ‘relative rotative efficiency $\eta_R$ as the factors relating the two conditions, according to the following equations (thrust equalization method is assumed, Taylor’s definition of wake fraction $W$ is used in (7)):

$$
\begin{aligned}
T_0 &= T (5) \\
Q_0 &= Q \eta_R (6) \\
V_0 &= V (1 - W) (7)
\end{aligned}
$$

where $T_0$, $V_0$, $Q_0$ denote the propeller thrust, effective speed of advance and torque of the propeller operating separate from the ship and, $T$, $V$, $Q$ the propeller thrust,
ship’s speed and torque of the propeller operating behind ship.

We use the following notation for the dimensionless variables for the propeller operation separate from the ship (open water or free running conditions):

$$
\begin{aligned}
J_0 &= V_0 / (n D) : advance coefficient, open-water (8) \\
k_{t0} &= T_0 / (\rho n^2 D^4) : thrust coefficient, open-water (9) \\
k_{q0} &= Q_0 / (\rho n^2 D^5): torque coefficient, open-water (10) \\
\eta_0 &= T_0 V_0 / (2 \pi n Q_0) = J_0 k_{t0} / (2 \pi k_{q0}): propeller efficiency, open-water (11)
\end{aligned}
$$

where $n$ is the propeller rotational velocity and $D$ the propeller diameter.

Behind-hull operation (self-propulsion) is modeled using the classical interaction parameters wake fraction $w$, thrust deduction $t$, and relative rotative efficiency $\eta_R$ (Birk, 2019).

- Wake fraction (Birk, 2019, Eq. (32.4)): $V_A = V(1-w)$
- Advance ratio for the open-water curves (Birk, 2019, Eq. (41.7)): $J = V_A/(nD)$

We do not have behind-hull thrust/torque measurements, so we reuse the open-water curves evaluated at the effective advance speed $V_A$:

$$
\begin{aligned}
T &\approx T_0 = \rho n^2 D^4 K_T(J) \\n
Q &\approx Q_0/\eta_R = \rho n^2 D^5 K_Q(J) / \eta_R
\end{aligned}
$$

Thrust deduction is applied on the ship force balance, not by modifying the resistance term:
- Birk (2019), Eq. (32.2)–(32.3): $T_E=(1-t)T$ and at steady self-propulsion $T_E = R_T$

Efficiencies:
- Open-water: $\eta_0 = J K_T/(2\pi K_Q)$ (Birk, 2019, Eq. (41.15); Molland et al., 2017, Eq. (12.2))
- Behind-hull: $\eta_B = (T V_A)/(2\pi n Q)$ and $\eta_R = \eta_B/\eta_0$ (Birk, 2019, Eq. (32.5) and Eq. (32.11))


In [1]:
import importlib.util
import json
import numpy as np
import pandas as pd
import sys
from pathlib import Path

sys.path.append(".")
sys.path.append("../hull")
sys.path.append("../../..")

from helper import (
    estimate_initial_values,
    load_boat_data,
)
from utils.optimization import load_model_params_from_json

# Load ESC and Motor models from sibling folders
esc_spec = importlib.util.spec_from_file_location("esc_model", Path("../esc/model.py"))
esc_model = importlib.util.module_from_spec(esc_spec)
esc_spec.loader.exec_module(esc_model)
ESC = esc_model.ESC

motor_spec = importlib.util.spec_from_file_location(
    "motor_model", Path("../motor/model.py")
)
motor_model = importlib.util.module_from_spec(motor_spec)
motor_spec.loader.exec_module(motor_model)
Motor = motor_model.SteadyStateMotor

hull_spec = importlib.util.spec_from_file_location(
    "hull_model", Path("../hull/hydrostatic_hull.py")
)
hull_model = importlib.util.module_from_spec(hull_spec)
hull_spec.loader.exec_module(hull_model)
HydrostaticHull = hull_model.HydrostaticHull

In [2]:
# Load blade geometry results from image (exported by blade_area_from_image.ipynb)
with open("propeller_blade_geometry.json", "r", encoding="utf-8") as f:
    blade_geom_json = json.load(f)

blade_geom = {
    "prop_D": blade_geom_json["prop_diameter_cm"] / 100,
    "prop_AEA0": blade_geom_json["AEA0"],
    "prop_c_07": blade_geom_json["chord_cm"] / 100,
}

In [3]:
hull_input_params = load_model_params_from_json(
    "../../../models/2020/hull/data/hull_input_params.json"
)

hull = HydrostaticHull(
    cog_x=hull_input_params["cog_x"],
    disp_mass=hull_input_params["disp_mass"],
    total_area=hull_input_params["total_area"],
)
hull_params = hull.get_params()

hull_LWL = hull_params["LWL"]
hull_BWL = hull_params["BWL"]
hull_M = hull_params["disp_mass"]
hull_S_water = hull_params["wet_surface_area_interp"]
hull_S_air = np.abs(hull_params["total_area"] - hull_params["wet_surface_area_interp"])
hull_C_B = hull_params['CB']

In [4]:
# Load model parameters
esc_params = load_model_params_from_json("../../../models/2020/esc/mam17_params.json")
motor_params = load_model_params_from_json(
    "../../../models/2020/motor/me0909_free_rotor_step_params.json"
)
generation_params = load_model_params_from_json(
    "../../../models/2020/generation/generation_params.json"
)

# Load boat dataset and select operating point near max input power
df = load_boat_data("../../../models/2020/boat_data_50ms.csv")

# Getting system operating point:
train_section = dict(
    start=pd.Timestamp("2020-02-02 10:02:29"), end=pd.Timestamp("2020-02-02 10:03:17")
)
df = df.loc[train_section["start"]:train_section["end"]]

display(df.describe())

,t,batt_v,batt_i,esc_d,motor_w,mppt1_i_in,mppt2_i_in,mppt3_i_in,mppt4_i_in,mppt1_v_in,mppt2_v_in,mppt3_v_in,mppt4_v_in,mppt1_v_out,mppt2_v_out,mppt3_v_out,mppt4_v_out,hull_u
count,49.00000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,338760.00000,32.645325,117.693732,0.996989,281.222785,2.260104,2.476675,2.597283,2.216375,29.302767,28.245920,29.534937,29.576455,33.056312,33.140823,33.100734,33.003016,4.832989
std,14.28869,0.249000,8.253931,0.008377,6.633825,0.125752,0.071235,0.071275,0.090065,0.379543,0.416612,0.334093,0.165402,0.248662,0.268374,0.231295,0.280844,0.037745
min,338736.00000,32.017942,107.304750,0.972930,261.882804,1.893110,2.357626,2.468912,2.072166,28.725602,27.684296,28.905592,29.205280,32.450204,32.530310,32.462448,32.349540,4.749363
25%,338748.00000,32.538551,112.171750,1.000000,276.224812,2.203965,2.426511,2.541021,2.142430,29.061628,28.031632,29.348333,29.502922,32.856321,33.028409,33.012237,32.844945,4.811037
50%,338760.00000,32.735782,114.324750,1.000000,283.463148,2.261849,2.465216,2.601234,2.190763,29.146662,28.142611,29.545227,29.545859,33.133745,33.208683,33.153394,33.061088,4.847370
75%,338772.00000,32.815540,120.634874,1.000000,286.373037,2.352032,2.526795,2.640161,2.312030,29.424306,28.251023,29.678610,29.619439,33.201521,33.270150,33.230209,33.164703,4.860791
max,338784.00000,32.953791,139.246250,1.000000,288.717603,2.451210,2.608212,2.741792,2.363298,30.174547,29.551682,30.396275,30.205811,33.498224,33.688881,33.538528,33.628880,4.880538


In [5]:
# This is our best approximation of the ESC's input current:

oth_p_in = 1.5 * 18
df["oth_i_in"] = oth_p_in / df["batt_v"]

for n in [1,2,3,4]:
    df[f"mppt{n}_p_in"] = df[f"mppt{n}_i_in"] * df[f"mppt{n}_v_in"]

    df[f"mppt{n}_p_out"] = generation_params['mppt_eta'] * df[f"mppt{n}_p_in"]
    df[f"mppt{n}_i_out"] = df[f"mppt{n}_p_out"] / df[f"mppt{n}_v_out"]

df["mppts_p_in"] = df["mppt1_p_in"] + df["mppt2_p_in"] + df["mppt3_p_in"] + df["mppt4_p_in"]
df["mppts_i_in"] = df["mppt1_i_in"] + df["mppt2_i_in"] + df["mppt3_i_in"] + df["mppt4_i_in"]

df["mppts_p_out"] = df["mppt1_p_out"] + df["mppt2_p_out"] + df["mppt3_p_out"] + df["mppt4_p_out"]
df["mppts_i_out"] = df["mppt1_i_out"] + df["mppt2_i_out"] + df["mppt3_i_out"] + df["mppt4_i_out"]
df["esc_i_in"] = df["batt_i"] + df["mppts_i_out"] - df["oth_i_in"]

In [6]:
# Get the average as the operating point:
op = df.mean(numeric_only=True)

esc_i_in = op["esc_i_in"]
batt_v = op["batt_v"]
esc_d = op["esc_d"]
motor_w = op["motor_w"]
hull_u = op["hull_u"]

display(op)

t              338760.000000
batt_v             32.645325
batt_i            117.693732
esc_d               0.996989
motor_w           281.222785
mppt1_i_in          2.260104
mppt2_i_in          2.476675
mppt3_i_in          2.597283
mppt4_i_in          2.216375
mppt1_v_in         29.302767
mppt2_v_in         28.245920
mppt3_v_in         29.534937
mppt4_v_in         29.576455
mppt1_v_out        33.056312
mppt2_v_out        33.140823
mppt3_v_out        33.100734
mppt4_v_out        33.003016
hull_u              4.832989
oth_i_in            0.827118
mppt1_p_in         66.204595
mppt1_p_out        61.725585
mppt1_i_out         1.867300
mppt2_p_in         69.951730
mppt2_p_out        65.219212
mppt2_i_out         1.968069
mppt3_p_in         76.713211
mppt3_p_out        71.523251
mppt3_i_out         2.160855
mppt4_p_in         65.552440
mppt4_p_out        61.117552
mppt4_i_out         1.852077
mppts_p_in        278.421976
mppts_i_in          9.550437
mppts_p_out       259.585601
mppts_i_out   

In [7]:
import numpy as np
from scipy.optimize import root  # or fsolve


def solve_op_q_load(
    *,
    batt_v_meas: float,
    esc_d: float,
    esc_i_in_meas: float,
    motor_params: dict,
    esc_params: dict,
    motor_w_guess: float,
):
    # Initial guesses
    w0 = float(motor_w_guess)
    # crude q guess from electrical power / omega (assume some efficiency)
    # avoid divide-by-zero if w0 ~ 0
    q0 = 0.7 * (batt_v_meas * max(esc_i_in_meas, 0.0)) / max(w0, 1e-3)

    def residuals(x):
        motor_w, motor_q_load = x

        # 1) ESC -> motor voltage guess (start with previous iterate’s implied vout)
        # We'll compute esc_v_out_hat using motor_i_hat, but motor_i_hat needs motor_v.
        # So do a tiny fixed-point inside: start with v = esc_d*batt_v then update once.
        motor_v = esc_d * batt_v_meas

        # One (or a few) refinement iterations usually helps convergence
        for _ in range(2):
            motor_inputs = np.array([motor_v, motor_q_load], dtype=float)
            motor_i_hat, motor_w_hat = Motor._outputs(
                None, None, motor_inputs, motor_params
            )

            esc_inputs = np.array([batt_v_meas, motor_i_hat, esc_d], dtype=float)
            esc_v_out_hat, esc_i_in_hat = ESC._outputs(
                None, None, esc_inputs, esc_params
            )

            motor_v = esc_v_out_hat  # enforce coupling for next refinement

        # Two equations:
        r1 = motor_w_hat - motor_w
        r2 = esc_i_in_hat - esc_i_in_meas
        return np.array([r1, r2], dtype=float)

    sol = root(
        residuals,
        x0=np.array([w0, q0], dtype=float),
        method="hybr",  # same family as fsolve
        tol=1e-10,
    )
    if not sol.success:
        raise RuntimeError(f"root() failed: {sol.message}")

    motor_w_sol, motor_q_load_sol = sol.x

    # Recompute “final” coupled outputs at the solution
    motor_v = esc_d * batt_v_meas
    for _ in range(3):
        motor_inputs = np.array([motor_v, motor_q_load_sol], dtype=float)
        motor_i_hat, motor_w_hat = Motor._outputs(
            None, None, motor_inputs, motor_params
        )

        esc_inputs = np.array([batt_v_meas, motor_i_hat, esc_d], dtype=float)
        esc_v_out_hat, esc_i_in_hat = ESC._outputs(None, None, esc_inputs, esc_params)
        motor_v = esc_v_out_hat

    # Compute power and efficiencies
    motor_p_in = motor_i_hat * motor_v
    motor_p_out = motor_q_load_sol * motor_w_hat
    motor_eta = motor_p_out / motor_p_in if motor_p_in != 0 else np.nan
    esc_i_out = motor_i_hat
    esc_p_out = esc_v_out_hat * esc_i_out
    esc_v_in = batt_v
    esc_p_in = esc_v_in * esc_i_in
    esc_eta = esc_p_out / esc_p_in if esc_p_in != 0 else np.nan

    return {
        "motor_w": float(motor_w_sol),
        "motor_q_load": float(motor_q_load_sol),
        "motor_v": float(motor_v),
        "motor_i": float(motor_i_hat),
        "motor_w": float(motor_w_hat),
        "motor_p_out": float(motor_p_out),
        "motor_p_in": float(motor_p_in),
        "motor_eta": float(motor_eta),
        "esc_v_out": float(esc_v_out_hat),
        "esc_i_in": float(esc_i_in_hat),
        "esc_p_out": float(esc_p_out),
        "esc_p_in": float(esc_p_in),
        "esc_eta": float(esc_eta),
        "esc_d": float(esc_d),
        "converged": bool(sol.success),
        "residuals": sol.fun.astype(float),
    }


motor_esc_operating_point = solve_op_q_load(
    batt_v_meas=float(batt_v),
    esc_d=float(esc_d),
    esc_i_in_meas=float(esc_i_in),
    motor_params=motor_params,
    esc_params=esc_params,
    motor_w_guess=float(motor_w),
)
motor_esc_operating_point

{'motor_w': 296.3577653670202,
 'motor_q_load': 12.559819680291806,
 'motor_v': 32.547034623725594,
 'motor_i': 121.96557442066118,
 'motor_p_out': 3722.2000938640017,
 'motor_p_in': 3969.61777357184,
 'motor_eta': 0.9376721654777322,
 'esc_v_out': 32.547034623725594,
 'esc_i_in': 124.71491496195776,
 'esc_p_out': 3969.61777357184,
 'esc_p_in': 4071.3589227540465,
 'esc_eta': 0.9750105183275308,
 'esc_d': 0.996989145976862,
 'converged': True,
 'residuals': array([ 0.00000000e+00, -2.84217094e-14])}

In [8]:
# Estimate propeller and chain performancea
in_to_m = 0.0254
radps_to_rpm = 60 / (2 * np.pi)

vals = estimate_initial_values(
    motor_rpm=motor_w * radps_to_rpm,
    batt_v=batt_v,
    esc_i_in=motor_esc_operating_point["motor_i"],
    motor_eta=motor_esc_operating_point["motor_eta"],
    esc_eta=motor_esc_operating_point["esc_eta"],
    trans_eta=0.95,
    trans_k=14 / 22,
    hull_u=hull_u,
    hull_L=hull_LWL,
    hull_B=hull_BWL,
    hull_M=hull_M,
    hull_S_water=hull_S_water,
    hull_S_air=hull_S_air,
    hull_C_B=hull_C_B,
    prop_D=blade_geom["prop_D"],
    prop_Z=3,
    prop_hub_D=0.06,
    prop_AEA0=blade_geom["prop_AEA0"],
    prop_PD=10.5 / 9,
    prop_total_m=0.66,
    prop_blade_thickness=0.005,
    prop_mat_rho=2700.0,
    prop_c_07=blade_geom["prop_c_07"],
    water_rho=1023,  # Estimated from public ocean dataset
    air_rho=1.1839,  # Estimated from public ocean dataset
    water_mu=0.0009268,  # Estimated from public ocean dataset
)
display(vals)

{'motor_rpm': 2685.4797772996712,
 'motor_n': 44.75799628832785,
 'motor_w': 281.22278465762,
 'motor_v': 32.64532493163265,
 'motor_i': 121.96557442066118,
 'motor_eta': 0.9376721654777322,
 'esc_eta': 0.9750105183275308,
 'trans_eta': 0.95,
 'trans_k': 0.6363636363636364,
 'esc_P_in': 3981.605807435708,
 'esc_P_out': 3882.1075420837965,
 'motor_P_shaft': 3640.14418560315,
 'trans_P_out': 3458.1369763229923,
 'system_eta_elec_to_prop': 0.9494456927139321,
 'system_eta_available_to_prop': 1.0931662076538655,
 'hull_u': 4.832989395737871,
 'hull_L': 5.116124628505016,
 'hull_B': 1.0152722270606231,
 'hull_M': 293.7,
 'hull_C_B': 0.40288607944614724,
 'hull_S_air': 4.188174709710495,
 'hull_S_water': 4.050226176680205,
 'hull_C_T': 0.010878369245660298,
 'hull_T': 0.13718999158735545,
 'hull_Fn': 0.682198532558467,
 'hull_W': 0.1514430397230736,
 'hull_T_ded': 0.09086582383384416,
 'hull_k_R': 0.6,
 'hull_eta_H': 1.0713885086387838,
 'prop_rpm': 1708.941676463427,
 'prop_n': 28.482361274

In [9]:
from utils.optimization import save_model_params_to_json

save_model_params_to_json("data/propulsive_initial_params.json", vals)